<a href="https://colab.research.google.com/github/jmelm93/semrush-api-backfill-site-data/blob/main/Backfill_SEMRush_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import json
import requests
import logging
import urllib
import pandas as pd
from datetime import datetime, date
import dateutil.relativedelta

today = date.today().strftime("%Y-%m-%d") 

In [29]:
apikey = "XXXXXXXXXXXXXXX"
columns = "Xn,Dn,Rk,Or,Ot,Oc,FKn,FPn"
domain_list=["layla.com","inseev.com"]
database="us"
end_date = today
start_date_months_back = 12

In [ ]:
shifted_timestamp = pd.Timestamp(today) - pd.DateOffset(months=start_date_months_back)
start_date = shifted_timestamp.strftime("%Y-%m-%d") 

date_list=pd.date_range(start_date,end_date, 
              freq='MS').strftime("%Y%m15").tolist()

date_list

In [31]:
# https://www.semrush.com/api-analytics/#rank

def get_report_data(apikey,domain,database,columns,date):
  """The get_report_data function takes the following parameters:

      Args:
        apikey: your API key
        domain: the domain you want to analyze
        database: The database to use. Can be "us", "fr", "de", "uk", "au", "ca", "it", "nl", "es", "br", "ru", "jp", "in", "mx", "cn", "tr", "be", "dk"
        columns: The columns you want to export.
        date: The date of the report.

      Returns:
        The report is being returned as a byte object.
          
  """
  report = requests.get(
      url="https://api.semrush.com",
      params={
          "type": "domain_rank",
          "key": apikey,
          "domain": domain,
          "database": database,
          "export_escape": 1,
          "export_decode": 1,
          "display_date": date.replace('-',''),
          "export_columns": columns
      }
  )
  logging.info(report.status_code)
  return report.content


In [32]:
def parse_response(data):
  """Parse the response from the server and return a list of dictionaries.
      
      Args:
        data: The data to be sent to the server.
      
      Returns:
        A list of dictionaries. Each dictionary is a row of data.
  """
  results = []
  data = data.decode('unicode_escape')
  lines = data.split('\r\n')
  lines = list(filter(bool, lines))
  columns = lines[0].split(';')

  for line in lines[1:]:
      result = {}
      for i, datum in enumerate(line.split(';')):
          result[columns[i]] = datum.strip('"\n\r\t')
      results.append(result)

  return results

In [34]:
results = []
for domain in domain_list:
  for date in date_list:
    data = get_report_data(apikey=apikey,database=database,domain=domain,columns=columns,date=date)
    parsed_data = parse_response(data=data)
    df = pd.DataFrame(parsed_data)
    df['date']=date
    results.append(df)


In [ ]:
results

In [ ]:
pd.concat(results)